## 导入数据及预处理

In [ ]:
! pip install pandas numpy matplotlib seaborn scikit-learn

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def preprocess_abundance(abu, level):
    abu = abu.T # 转置
    abu = abu.loc[:,abu.sum(axis=0) > 0] # 去除全为0的特征
    abu.columns = abu.columns.str.split(';', expand=True) # 将分类信息拆分为多列
    abu.columns = abu.columns.get_level_values(level) # 选择指定层级的分类信息
    abu = abu.loc[:,abu.columns.notnull()] # 去除分类信息为空的特征
    abu = abu.groupby(abu.columns, axis=1).sum() # 按分类信息聚合\
    return abu
    

导入数据

In [ ]:
X_train = pd.read_csv('data/SourceCM.csv', index_col=0)
X_test = pd.read_csv('data/QueryCM.csv', index_col=0)
y_test = pd.read_csv('data/QueryLabel.csv', index_col=0)
y_train = pd.read_csv('data/SourceLabel.csv', index_col=0)

In [ ]:
X_train.describe()

丰度表预处理

In [ ]:
X_train = preprocess_abundance(X_train, 5)  # 保留到属
X_test = preprocess_abundance(X_test, 5)  # 保留到属

In [ ]:
X_test = pd.concat([pd.DataFrame(columns=X_train.columns), X_test], join='outer', axis=0).fillna(0)  # 特征对齐
X_test = X_test[X_train.columns]

X_train = X_train.div(X_train.sum(axis=1), axis=0)  # 相对丰度
X_test = X_test.div(X_test.sum(axis=1), axis=0) # 相对丰度

In [ ]:
X_train.head()

利用训练集的丰度表进行标准化

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

标签预处理

In [ ]:
y_train = y_train['Env'].str.split(':', expand=True)    # 样本来源分层
y_train = y_train.drop(columns=[0]) # 去除root
y_test = y_test['Env'].str.split(':', expand=True)    # 样本来源分层
y_test = y_test.drop(columns=[0]) # 去除root

In [ ]:
y_train.head()

## 模型构建

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def construct_model(X_train, y_train):
    le = LabelEncoder()
    le.fit_transform(y_train)   # 将分类标签编码为数字
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(X_train, y_train)
    return clf, le

对最后一层建模

In [ ]:
model, le = construct_model(X_train, y_train.iloc[:, -1])

在测试集上评估

In [ ]:
y_score = model.predict_proba(X_test)
y_true = le.transform(y_test.iloc[:, -1])

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def roc_auc_calculate(y_query, y_proba):
    classes = np.unique(y_query)
    Nclasses = len(classes)
    y_test = np.eye(Nclasses)[y_query]

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(Nclasses):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_proba[:, i])
        roc_auc[i] = float(format(auc(fpr[i], tpr[i]), '.5f'))
    
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(Nclasses)]))#数组拼接得到fpr的矩阵

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)  #构造数字都为0的矩阵，为做平均做准备
    for i in range(Nclasses):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= Nclasses

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = float(format(auc(fpr["macro"], tpr["macro"]), '.5f'))
    

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_proba.ravel())
    roc_auc["micro"] = float(format(auc(fpr["micro"], tpr["micro"]), '.5f'))
    
    roc_auc["ovr"] = float(format(roc_auc_score(y_query, y_proba, multi_class='ovr'), '.5f'))
    roc_auc["ovo"] = float(format(roc_auc_score(y_query, y_proba, multi_class='ovo'), '.5f'))
    return roc_auc, fpr, tpr

In [ ]:
roc_auc, fpr, tpr = roc_auc_calculate(y_true, y_score)

## 可视化

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig = sns.lineplot(x=fpr['micro'], 
                   y=tpr['micro'], 
                   label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc['macro']))